__BIOBSS - PPG Signal Processing__

_This notebook includes guidelines to help using BIOBSS for PPG signal processing and feature extraction._

In [1]:
#Import BIOBSS and the other required packages

import biobss
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#help(biobss)

# Table of Contents
1.[PPG Sample Data](#sampledata)<br>
2.[PPG Signal Filtering](#ppg_pre)<br>
3.[PPG Peak Detection](#ppg_peak)<br>
4.[PPG Signal Quality Assessment](#ppg_sqa)<br>
5.[PPG Feature Extraction](#ppg_features)<br>

### __PPG Sample Data__
<a id="sampledata"></a>

In [2]:
#Load the sample data. The sample data file contains 100 10-s PPG segments. The samling rate of the segments is 64 Hz. 

data_dir=r'C:\Users\ipek.karakus\Desktop\biobss\sample_data'

filename='ppg_sample_data.csv'

data=pd.read_csv(os.path.join(data_dir,filename), header=None)
sig=np.asarray(data.iloc[0,:]) #The first segment is selected to be used as sample signal. 
fs=64
L=10

### __PPG Signal Preprocessing__
<a id="ppg_pre"></a>

BIOBSS library provides a filtering function based on Scipy. This function can be used to implement Butterworth filter by defining the filter parameters. 

In [4]:
#or defining the parameters.

f_sig= biobss.preprocess.filter_signal(sig,fs,'bandpass',N=2,f_lower=0.5,f_upper=5)

Additionally, pre-defined filter parameters can be used for each specific signal type. In order to use this option, signal_type should be selected as 'PPG'. 

In [3]:
#PPG signals can be filtered using the method below using pre-determined filter parameters

filtered_ppg=biobss.preprocess.filter_signal(sig, fs, signal_type='PPG')

c:\Users\ipek.karakus\.conda\envs\biolib\lib\site-packages\biobss\ppgtools\ppg_filter.py:12: UserWarning: Default parameters will be used for filtering. 2th order None bandpass filter with f1=0.5 and f2=5.
  warnings.warn(f"Default parameters will be used for filtering. {N}th order {method} {filter_type} filter with f1={f_lower} and f2={f_upper}.")


### __PPG Peak Detection__
<a id="ppg_peak"></a>

In [5]:
#Signal peaks can be detected as below

info=biobss.preprocess.peak_detection(sig,fs,'peakdet',delta=0.01)

locs_peaks=info['Peak_locs']
peaks=info['Peaks']
locs_onsets=info['Trough_locs']
onsets=info['Troughs']

#and the results can be corrected considering the order of peaks and onsets
info=biobss.ppgtools.peak_control(locs_peaks,peaks,locs_onsets,onsets)

locs_peaks=info['Peak_locs']
peaks=info['Peaks']
locs_onsets=info['Trough_locs']
onsets=info['Troughs']

In [7]:
#If needed, first and second derivatives of the signal can be calculated using the method below.
info=biobss.preprocess.signal_differentiate.differentiate_signal(sig)
vpg_sig=info['VPG']
apg_sig=info['APG']

### __PPG Signal Quality Assessment__
<a id="ppg_sqa"></a>

In [8]:
#Clipping and flatline detection can be done using the methods below
#For clipping detection
info=biobss.ppgtools.detect_flatline_clipping(sig,threshold=1.0, clipping=True)


In [9]:
#For flatline detection
info=biobss.ppgtools.detect_flatline_clipping(sig,threshold=0.0001,flatline=True, duration=10)

In [10]:
#The peaks should be detected for the following steps

info=biobss.preprocess.peak_detection(sig,fs,'peakdet',delta=0.01)
locs_peaks=info['Peak_locs']
peaks=info['Peaks']
locs_onsets=info['Trough_locs']
onsets=info['Troughs']

In [11]:
#and the results should be corrected considering the order of peaks and onsets
info=biobss.ppgtools.peak_control(locs_peaks,peaks,locs_onsets,onsets)
locs_peaks=info['Peak_locs']
peaks=info['Peaks']
locs_onsets=info['Trough_locs']
onsets=info['Troughs']

In [12]:
#Using the peak locations, the physiological and morphological limits can be checked

info=biobss.ppgtools.check_phys(locs_peaks,fs)
info=biobss.ppgtools.check_morph(locs_peaks,peaks,locs_onsets,onsets,fs)

In [13]:
#Template matching method can be applied using the method below.

info=biobss.ppgtools.template_matching(sig,locs_peaks)

### __PPG Feature Extraction__
<a id="ppg_features"></a>

In [14]:
#Cycle-based features can be calculated
ppg_cycle=biobss.ppgtools.ppg_features.from_cycles(sig,locs_peaks,peaks,locs_onsets,onsets,fs,feature_types=['Time','Stat'],prefix='ppg')


In [16]:
#Segment-based features can be calculated
ppg_segment=biobss.ppgtools.ppg_features.from_segment(sig,fs,feature_types=['Time','Stat','Freq'], prefix='ppg')